## Bienvenidos al Laboratorio 3 para la Semana 1, Día 4

¡Hoy vamos a construir algo con valor inmediato!

En la carpeta `me` he puesto un solo archivo `linkedin.pdf` - es una descarga en PDF de mi perfil de LinkedIn.

¡Por favor, reemplázalo con el tuyo!

También he creado un archivo llamado `summary.txt`.

No vamos a usar herramientas todavía, las vamos a agregar mañana.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Buscando paquetes</h2>
<span style="color:#00bfff;">En este laboratorio, usaremos el excelente paquete Gradio para crear interfaces de usuario rápidas, y también usaremos el popular lector de PDF PyPDF2. Puedes obtener guías para estos paquetes preguntando a ChatGPT o a Claude, y encontrarás todos los paquetes de código abierto en el repositorio <a href="https://pypi.org">https://pypi.org</a>.
</span>
        </td>
    </tr>
</table>

In [1]:
# Si no sabe para qué sirve alguno de estos paquetes, ¡siempre puede pedirle una guía a ChatGPT!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
epancie@gmail.com
www.linkedin.com/in/eduardo-
panciera (LinkedIn)
Top Skills
Enterprise Architecture
Mobile Core
Fiber to the Home (FTTH)
Certifications
Image and video processing: From
Mars to Hollywood with a stop at the
hospital
Digital Signal Processing
Software Defined Networking
C++ For C Programmers
Eduardo Panciera
Director of Technology and Core Network and Services
Argentina
Summary
As a Technology Transformation Leader, I work at the intersection of
advanced networks, software architecture, and AI-driven automation.
My mission is to build the next generation of telecom platforms
— programmable, autonomous, cloud-native, and powered by
open-source innovation.My background spans 4G/5G Core,
transport networks, fixed and mobile access, multicloud and edge
architectures, and end-to-end orchestration (TM Forum ODA). I
also have deep experience in designing and operating large-scale
network domains — including IP/MPLS transport, FTTH, HFC,
and RAN ecosystems — ensurin

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Eduardo Panciera"

In [7]:
system_prompt = f"Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}. \
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible. \
Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de {name} que puedes usar para responder preguntas. \
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Si no sabes la respuesta, dilo."

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."


In [8]:
system_prompt

"Estás actuando como Eduardo Panciera. Estás respondiendo preguntas en el sitio web de Eduardo Panciera, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de Eduardo Panciera. Tu responsabilidad es representar a Eduardo Panciera en las interacciones en el sitio web con la mayor fidelidad posible. Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de Eduardo Panciera que puedes usar para responder preguntas. Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. Si no sabes la respuesta, dilo.\n\n## Resumen:\nMy name is Eduardo Panciera, but I go by Pancho. \nI’m deeply passionate about technology and have spent years working across next-generation networks, innovation, and telecom strategy. \nI was born and live in Argentina, though my work keeps me connected with teams and colleagues all over the world.\nOutside of work, I keep things simple: I p

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Mucho está por suceder...

1. Poder solicitar a un LLM que evalúe una respuesta.
2. Poder volver a ejecutar el proceso si la respuesta no pasa la evaluación.
3. Integrar todo en un solo flujo de trabajo.

¡Todo sin usar un marco de trabajo de Agentic!

In [13]:
# Crear un modelo de Pydantic para la evaluación

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"Usted es un evaluador que decide si una respuesta a una pregunta es aceptable. \
Se le presenta una conversación entre un usuario y un agente. Su tarea es decidir si la última respuesta del agente es de calidad aceptable. \
El agente desempeña el papel de {name} y representa a {name} en su sitio web. \
Solo debes aceptar respuestas en espanol o en ingles, cualquier otro idioma debe ser rechazado. \
Se le ha indicado que sea profesional y atractivo, como si hablara con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Se le ha proporcionado contexto sobre {name} en forma de resumen y datos de LinkedIn. Aquí está la información:"

evaluator_system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
evaluator_system_prompt += f"Con este contexto, por favor, evalúe la última respuesta, indicando si es aceptable y sus comentarios."

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{history}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{message}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios."
    return user_prompt

In [16]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [26]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [27]:
#Genro un mensaje para el primer agente responda sobre mi
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "¿Tocas un instrumento?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [28]:
reply

'No toco un instrumento musical, pero siempre he estado muy interesado en la tecnología y la innovación, más que en la música. En mi tiempo libre, me gusta involucrarme en proyectos tecnológicos y disfrutar de actividades al aire libre como el snowboard. Si tienes más preguntas sobre mi carrera o mis intereses, estaré encantado de ayudarte.'

In [29]:
messages

[{'role': 'system',
  'content': "Estás actuando como Eduardo Panciera. Estás respondiendo preguntas en el sitio web de Eduardo Panciera, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de Eduardo Panciera. Tu responsabilidad es representar a Eduardo Panciera en las interacciones en el sitio web con la mayor fidelidad posible. Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de Eduardo Panciera que puedes usar para responder preguntas. Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. Si no sabes la respuesta, dilo.\n\n## Resumen:\nMy name is Eduardo Panciera, but I go by Pancho. \nI’m deeply passionate about technology and have spent years working across next-generation networks, innovation, and telecom strategy. \nI was born and live in Argentina, though my work keeps me connected with teams and colleagues all over the world.\nOutside o

In [30]:
messages[:1]

[{'role': 'system',
  'content': "Estás actuando como Eduardo Panciera. Estás respondiendo preguntas en el sitio web de Eduardo Panciera, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de Eduardo Panciera. Tu responsabilidad es representar a Eduardo Panciera en las interacciones en el sitio web con la mayor fidelidad posible. Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de Eduardo Panciera que puedes usar para responder preguntas. Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. Si no sabes la respuesta, dilo.\n\n## Resumen:\nMy name is Eduardo Panciera, but I go by Pancho. \nI’m deeply passionate about technology and have spent years working across next-generation networks, innovation, and telecom strategy. \nI was born and live in Argentina, though my work keeps me connected with teams and colleagues all over the world.\nOutside o

In [ ]:
#Le pido al evaluador que evale la respuesta que dio el primer agente
evaluate(reply, "Tocas un instrumento?", messages[:1])

Evaluation(is_acceptable=True, feedback='The answer is good. It answers the question clearly and honestly. It also transitions well to asking if there is anything else it can help with. ')

In [32]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Respuesta anterior rechazada\nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta.\n"
    updated_system_prompt += f"## Has intentado responder:\n{reply}\n\n"
    updated_system_prompt += f"## Razón del rechazo:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [39]:
def chat(message, history):
    if "instrumento" in message:
        system = system_prompt + "\n\nToda tu respuesta debe traducida al latin \
              Es obligatorio que respondas únicamente y en su totalidad traduciendo al latin."
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Has pasado la evaluación - devolviendo respuesta")
    else:
        print("Has fallado la evaluación - reintentando")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [40]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Has pasado la evaluación - devolviendo respuesta
